In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torchtext.vocab import GloVe
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import SubsetRandomSampler,DataLoader
import torch.optim as optim
import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker



TRAINABLE_EMBEDDINGS = False
EMBED_DIM = 300 
ENC_BIDIRECTIONAL = True
ENC_BIDIRECTIONAL_FACTOR = 2 if ENC_BIDIRECTIONAL else 1
ENC_HIDDEN_DIM = 256
DEC_HIDDEN_DIM  = 256
ENC_OUTPUT_DIM =  DEC_HIDDEN_DIM
DEC_EMBED_DIM = 300
NUM_EPOCHS = 300

BATCH_SIZE = 128

device_cpu = torch.device('cpu')
device_fast = torch.device('cpu')

if torch.has_mps:
    device_fast = torch.device('mps')
elif torch.has_cuda:
    device_fast = torch.device('cuda')


output_index_to_word = {}
output_vocab = {}

for i in range(0,10):
    output_vocab[str(i)] = i
    output_index_to_word[i] = str(i)

output_vocab['-'] = 10
output_index_to_word[10] = '-'
output_vocab['<sos>'] = 11
output_index_to_word[11] = '<sos>'
output_vocab['<eos>'] = 12
output_index_to_word[12] = '<eos>'

glove = GloVe()

/data1/depressedcoder/anaconda3/envs/pytorchenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:

def get_training_data(filename='./Assignment4aDataset.txt',glove=glove):
    
    f = open(filename,'r')
    dataset = []
    for line in f.readlines():       
         
        nl_date , out_date = line.split(',')
        nl_date = nl_date.replace("\'","").strip()
        out_date = out_date.replace("\'","").strip()

        if "/" in nl_date:
            #number_list = list(nl_date)
            #nl_date = " ".join(number_list)
            split_on_slash = nl_date.split("/")
            nl_date = " / ".join(split_on_slash)

        nl_date = nl_date.lower()

        embeddings = []
        for word in nl_date.split(' '):
            if word == '':
                continue
            if not TRAINABLE_EMBEDDINGS:
                embeddings.append(glove[word])
            else:
                if word not in glove.stoi:
                    embeddings.append(glove.stoi['unk'])
                else:
                    embeddings.append(glove.stoi[word])
            
            #embeddings.append(torch.tensor(fasttext_model.get_word_vector(word)))
        current_inp_length = len(embeddings)
        if not TRAINABLE_EMBEDDINGS:
            embeddings = torch.stack(embeddings)
        else:
            embeddings = torch.tensor(embeddings)

        target = []
        target.append(output_vocab['<sos>'])

        for character in list(out_date):
            target.append(output_vocab[character])
        
        target.append(output_vocab['<eos>'])

        dataset.append({'in' : embeddings,'in_length' : current_inp_length,'out' : target})
    return dataset


class TranslationDataset(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


def collate_function(batch_data):
    inputs = [b['in'] for b in batch_data]
    in_lengths = [b['in_length'] for b in batch_data]
    out = torch.tensor([b['out'] for b in batch_data])
    inputs = pad_sequence(inputs,batch_first=True)
    return {'src': inputs, 'src_length' : in_lengths, 'trg' : out}

In [19]:
train_dataset = TranslationDataset(get_training_data())

train_idx,valid_idx = train_test_split(np.arange(len(train_dataset)), 
    test_size=0.2,
    shuffle= True,
    random_state=0
)

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_dataloader = DataLoader(train_dataset,BATCH_SIZE,sampler=train_sampler,collate_fn=collate_function)
valid_dataloader = DataLoader(train_dataset,BATCH_SIZE,sampler=valid_sampler,collate_fn=collate_function)

In [20]:
class Encoder(nn.Module):

    def __init__(self,embed_dim = EMBED_DIM,enc_hidden_dim = ENC_HIDDEN_DIM,enc_output_dim = ENC_OUTPUT_DIM,NUM_LAYERS=1,enc_bidirectional=ENC_BIDIRECTIONAL,dropout=0.3):
        super().__init__()
        
        #self.embedding_layer = nn.Embedding(vocab_size,EMBED_DIM)
        self.rnn = nn.GRU(embed_dim,enc_hidden_dim, num_layers = NUM_LAYERS ,batch_first= True ,bidirectional=enc_bidirectional)

        if TRAINABLE_EMBEDDINGS:
            self.embedding_layer = nn.Embedding.from_pretrained(glove.vectors,freeze=False)

        # ENCODER_OUTPUT_DIM = DECODER_HIDDEN_SIZE
        self.fc = nn.Linear(2*enc_hidden_dim,enc_output_dim) 

        self.fc_out = nn.Linear(enc_output_dim,1)
        nn.init.xavier_uniform_(self.fc.weight)
        self.dropout = nn.Dropout(dropout)

    def forward(self,inp,inp_len):
        
        if TRAINABLE_EMBEDDINGS:
            embedded_input = self.embedding_layer(inp)
        else:
            embedded_input = inp   # [batch_size, input_seq_length, embed_dim ]
        
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedded_input,inp_len,batch_first=True,enforce_sorted=False)
        packed_output , hidden = self.rnn(packed_embedding)  # hidden = [D*num_layers, batch_size , hidden_dim ]
        outputs, _  = nn.utils.rnn.pad_packed_sequence(packed_output,batch_first=True)  # [batch_size, inp_seq_length, hidden_dim]
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)))  # [batch_size, decoder_hidden_size]
        return outputs,hidden

In [21]:
class Attention(nn.Module):
    def __init__(self,enc_hidden_dim, dec_hidden_dim):
        super().__init__()
        
        self.attn = nn.Linear(enc_hidden_dim+dec_hidden_dim,dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim,1)

    def forward(self,hidden,encoder_outputs, encoder_length_mask):
        
        # encoder_outputs = [batch_size,seq_length, enc_hidden_dim][2*ENCODER_HIDDEN_DIM or ENCODER_HIDDEN_DIM]
        # hidden = [batch_size,  dec_hidden_dim]
        # encoder_length_mask = [batch_size, seq_length]

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        h = hidden.unsqueeze(1).repeat(1,src_len,1)  # h = [batch_size,seq_length,dec_hidden_dim]
        energy = torch.tanh(self.attn(torch.cat((h,encoder_outputs),dim=2)))  #[batch_size,seq_length,dec_hidden_dim]
        attention_scores = self.v(energy).squeeze(2)  # attention_scores = [batch_size , seq_length ]
        attention_scores = attention_scores.masked_fill(encoder_length_mask==1, -1e10)   # Fill padding tokens with a lower value
        attention_scores = F.softmax(attention_scores,dim=1)
        return attention_scores

In [22]:
def create_mask(src_lengths,max_src_length):

        src_mask = torch.zeros((len(src_lengths),max_src_length),dtype=torch.int64)
        for i in range(len(src_lengths)):

            src_mask[i,src_lengths[i]:] = 1
        return src_mask

enc = Encoder(30,30,15,10)

inp = torch.randn((3,20,30))
inp_len = [20 for i in range(3)]
outputs, hidden = enc(inp,inp_len)
outputs

In [23]:
class Decoder(nn.Module):

    def __init__(self,vocab_size,enc_hidden_dim,dec_hidden_dim,dec_output_dim,emb_dim):
        
        # enc_hidden_dim = 2*ENCODER_HIDDEN_DIM or ENCODER_HIDDEN_DIM

        super().__init__()
        
        self.vocab_size = vocab_size
        self.attention = Attention(enc_hidden_dim,dec_hidden_dim)
        self.embedding_layer = nn.Embedding(vocab_size,emb_dim)
        self.rnn = nn.GRU(enc_hidden_dim + emb_dim,dec_hidden_dim,batch_first = True)

        self.fc_out = nn.Linear(enc_hidden_dim + emb_dim + dec_hidden_dim , dec_output_dim)
        #self.emb_dropout = nn.Dropout(0.3)
        #self.fc_tilde = nn.Linear(enc_hidden_dim + emb_dim + dec_hidden_dim , dec_hidden_dim)
        #self.max_out_layer = nn.MaxPool1d(kernel_size=2)
        #self.W0 = nn.Linear(dec_hidden_dim>>1,emb_dim)
        #self.prob_out = nn.Linear(emb_dim,vocab_size)
        #self.prob_out.weight = self.embedding_layer.weight
        

    def forward(self,input,hidden,encoder_outputs,encoder_length_mask):
            # encoder outputs =  batch_size , seq_len , encoder_output_dim
            # hidden = batch_size , hidden_dim
            # input = batch_size
            
            input = input.unsqueeze(0) # [1,batch_size]
            embedded = self.embedding_layer(input)
            #embedded = self.emb_dropout(self.embedding_layer(input)) # [1,batch_size,embed_dim]

            embedded = embedded.permute(1,0,2) #[ batch_size, seq_length=1, embed_dim ]

            attention_vector = self.attention(hidden,encoder_outputs,encoder_length_mask) # [ batch_size , seq_length ]
            attention_vector = attention_vector.unsqueeze(1) # [batch_size , 1 , seq_length ]

            weighted = torch.bmm(attention_vector,encoder_outputs) # [ batch_size, 1, encoder_output_dim]
            #weighted = weighted.permute(1,0,2) #[1 , batch_size , encoder_output_dim]


            rnn_input = torch.cat((embedded,weighted),dim=2) #[batch_size, seq_length=1, encoder + decoder]

            out,h = self.rnn(rnn_input,hidden.unsqueeze(0)) # consider only a single layer (1.) so unsqueeze(0)

            # out = [batch_size, seq_length = 1, decoder_hidden_out (bidirectional)]
            # hidden = [D*num_layers,batch_size, decoder_hidden_out]


            embedded = embedded.squeeze(1)  # [batch_size,embed_dim]
            out = out.squeeze(1)    #[batch_size, decoder_hidden_out] # Have to change if the number of layers is changed to more than 1
            weighted = weighted.squeeze(1)  # [batch_size,encoder_output_dim] 
            prediction = self.fc_out(torch.cat([embedded,out,weighted],dim=1)) #[batch_size, decoder_output_dim]
            
            
            #prediction = F.softmax(self.fc_out(torch.cat([embedded,out,weighted],dim=1)),dim=1) #[batch_size, decoder_output_dim]
            #t_tilde =  self.fc_tilde(torch.cat([embedded,out,weighted],dim=1)) # [batch_size, decoder_hidden_dim]
            #t_tilde = self.max_out_layer(t_tilde.unsqueeze(1)) #[batch_size,1,decoder_hidden_dim/2]
            #t_tilde = t_tilde.squeeze(1)

            #inter_step  = self.W0(t_tilde) # [ batch_size , dec_embeddim]
            #prediction = self.prob_out(inter_step) #[  batch_size , vocab_size]
            #prediction = F.softmax(prediction,dim=1)
            return prediction, h.squeeze(0), attention_vector # Reduce the number of layers

In [24]:
class TranslationModel(nn.Module):

    def __init__(self,
                input_embed_dim = EMBED_DIM,
                encoder_hidden_dim = ENC_HIDDEN_DIM,
                encoder_hidden_output = ENC_OUTPUT_DIM,
                enc_num_layers = 1,
                enc_bidirectional = ENC_BIDIRECTIONAL,

                dec_vocab_size = len(output_vocab),
                dec_embed_dim = DEC_EMBED_DIM,
                dec_hidden_dim  =DEC_HIDDEN_DIM,
                device_train = device_cpu
        ):
        
        super().__init__()

        self.encoder = Encoder(input_embed_dim,encoder_hidden_dim,encoder_hidden_output,enc_num_layers,enc_bidirectional=enc_bidirectional)
        enc_bidirectional_factor = 2 if enc_bidirectional else 1
        self.decoder = Decoder(dec_vocab_size,enc_bidirectional_factor*encoder_hidden_dim,dec_hidden_dim=dec_hidden_dim,dec_output_dim=dec_vocab_size,emb_dim=dec_embed_dim)
        self.device_train = device_train


    def create_mask(self, src_lengths,max_src_length):

        src_mask = torch.zeros((len(src_lengths),max_src_length),dtype=torch.int64)
        for i in range(len(src_lengths)):

            src_mask[i,src_lengths[i]:] = 1
        return src_mask
        
    def forward(self,source,source_len,target,teacher_forcing_ratio = 0.0):
        #   source = [batch_size, max_src_len]
        #   source_len = [length of sentence in the batch]
        #   target = [batch_size,traget_length]
        #   teacher_forcing_ratio = probability to use teacher forcinbg

        batch_size = source.shape[0]
        target_length = target.shape[1]
        target_vocab_size = self.decoder.vocab_size
        outputs= torch.zeros(batch_size,target_length,target_vocab_size).to(self.device_train)
        encoder_outputs , hidden = self.encoder(source,source_len)

        inp = target[:,0]        
        enc_mask = self.create_mask(source_len,int(encoder_outputs.shape[1]))
        enc_mask = enc_mask.to(self.device_train)
        for t in range(1,target_length):
            decoder_output, hidden, attention_vector =  self.decoder(inp,hidden,encoder_outputs,enc_mask)

            outputs[:,t,:] = decoder_output # batch_size, vocab_size
            teacher_force = random.random() < teacher_forcing_ratio 

            top1 = decoder_output.argmax(1)

            inp = target[:,t] if teacher_force else top1
        
        return outputs

In [25]:
def apply_weights(model : TranslationModel):
    
    for name,param in model.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data,mean=0,std=0.01)
            #nn.init.xavier_uniform_(param.data)
        else:
            nn.init.constant_(param.data,0)
    
    
    #nn.init.xavier_uniform_(model.encoder.fc_out.weight)
    #nn.init.xavier_uniform_(model.decoder.attention.attn.weight)

    nn.init.normal_(model.decoder.attention.attn.weight, mean=0, std=0.001)

    

t = TranslationModel()
apply_weights(t)
batch_data = next(iter(train_dataloader))

criterion = nn.CrossEntropyLoss(ignore_index=output_vocab['<eos>'])
optimizer = optim.Adam(t.parameters(),lr=0.001)



model_output = t(batch_data['src'],batch_data['src_length'],batch_data['trg'])


model_out_reshaped = model_output[1:].view(-1,model_output.shape[-1])
reshaped_target = batch_data['trg'][1:].view(-1)
loss_value = criterion(model_out_reshaped,reshaped_target)
loss_value.backward()
nn.utils.clip_grad_norm_(t.parameters(),5)
optimizer.step()
print(loss_value.item())

In [26]:
from torch.utils.tensorboard import SummaryWriter
from datetime import  datetime

import os

def train_model(model,num_epochs,train_loader,valid_loader,optimizer,criterion,checkpoint_name='translation_model.pth',device_train = device_cpu):
    
    current_datetime = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    tensorboard_name =  checkpoint_name + current_datetime
    writer = SummaryWriter('runs/' + tensorboard_name)

    best_validation_loss = 1000.0
    model = model.to(device_train)
    for e in range(num_epochs):

        training_loss = 0.0
        model.train()
        for i,batch in enumerate(train_loader):

            source, source_length, target = batch['src'], batch['src_length'], batch['trg']

            source = source.to(device_train)
            target = target.to(device_train)
            optimizer.zero_grad()

            # model_output = [batch_size, output_seq_length,vocab_size]
            model_output  = model(source,source_length,target)


            source = source.to(device_cpu)
            target = target.to(device_cpu)
            model_output = model_output.to(device_cpu)
            
            model_out_reshaped = model_output[1:].view(-1,model_output.shape[-1]) 
            #print(model_out_reshaped.shape)
            reshaped_target = target[1:].view(-1)
            #print(reshaped_target.shape)

            loss_value = criterion(model_out_reshaped,reshaped_target)
            loss_value.backward()
            nn.utils.clip_grad_norm_(model.parameters(),5)

            optimizer.step()
            training_loss += loss_value.item()
        
        print("Epoch " + str(e) + " Training Loss Value = " + str(training_loss/len(train_loader)))
        writer.add_scalars('Train/Loss vs Epoch',{'train' :training_loss/len(train_loader)},e)

        model.eval()
        validation_loss = 0.0

        with torch.no_grad():

            for i, batch in enumerate(valid_loader):
                source, source_length, target = batch['src'], batch['src_length'], batch['trg']

                source = source.to(device_train)
                target = target.to(device_train)

                model_output  = model(source,source_length,target,0)


                source = source.to(device_cpu)
                target = target.to(device_cpu)
                model_output = model_output.to(device_cpu)
                
                model_out_reshaped = model_output[1:].view(-1,model_output.shape[-1])
                reshaped_target = target[1:].view(-1)
                loss_value = criterion(model_out_reshaped,reshaped_target)

                validation_loss += loss_value.item()
        averaged_validation_loss = validation_loss/len(valid_loader)
        writer.add_scalars('Train/Loss vs Epoch',{'valid' :averaged_validation_loss},e)

        print("Epoch " + str(e) + " Validation Loss Value = " + str(averaged_validation_loss))
        print("")
        if (averaged_validation_loss <= best_validation_loss):
            best_validation_loss = averaged_validation_loss
            torch.save(model.state_dict(),checkpoint_name)
    

In [27]:
t = TranslationModel(device_train=device_fast)
apply_weights(t)
train_model(t,750,train_dataloader,valid_dataloader,optim.Adam(t.parameters(),lr=0.001),nn.CrossEntropyLoss(),device_train= device_fast,checkpoint_name='translation_model_11')

Epoch 0 Training Loss Value = 1.1530088188648224
Epoch 0 Validation Loss Value = 0.5815817988108075

Epoch 1 Training Loss Value = 0.45360297453403475
Epoch 1 Validation Loss Value = 0.3678397962025234

Epoch 2 Training Loss Value = 0.3087123005390167
Epoch 2 Validation Loss Value = 0.2678949269983504

Epoch 3 Training Loss Value = 0.24842149513959885
Epoch 3 Validation Loss Value = 0.24005483611235542

Epoch 4 Training Loss Value = 0.2338920983672142
Epoch 4 Validation Loss Value = 0.23280652908105698

Epoch 5 Training Loss Value = 0.4828983426094055
Epoch 5 Validation Loss Value = 0.24144677273810855

Epoch 6 Training Loss Value = 0.23431067776679992
Epoch 6 Validation Loss Value = 0.23017132471478174

Epoch 7 Training Loss Value = 0.22913124603033067
Epoch 7 Validation Loss Value = 0.22751961388285197

Epoch 8 Training Loss Value = 0.227423486828804
Epoch 8 Validation Loss Value = 0.23006540584185767

Epoch 9 Training Loss Value = 0.22567995297908783
Epoch 9 Validation Loss Value = 

KeyboardInterrupt: 

In [28]:
def translate_sentence(sentence, model, max_len = 10,glove=glove):

    model.eval()
    nl_date = sentence.replace("\'","").strip()
    
    nl_date = nl_date.lower()

    if "/" in nl_date:
        #number_list = list(nl_date)
        #nl_date = " ".join(number_list)
        split_on_slash = nl_date.split("/")
        nl_date = " / ".join(split_on_slash)

        
    embeddings = []
    for word in nl_date.split(' '):
        if not TRAINABLE_EMBEDDINGS:
            embeddings.append(glove[word])
        else:
            if word not in glove.stoi:
                embeddings.append(glove.stoi['unk'])
            else:
                embeddings.append(glove.stoi[word])
        #embeddings.append(torch.tensor(fasttext_model.get_word_vector(word)))


    current_inp_length = len(embeddings)
    if not TRAINABLE_EMBEDDINGS:
        inp_embeddings = torch.stack(embeddings)
    else:
        inp_embeddings = torch.tensor(embeddings)
    inp_embeddings = inp_embeddings.unsqueeze(0)

    with torch.no_grad():
       
        encoder_outputs, hidden = model.encoder(inp_embeddings, [current_inp_length])
        mask = model.create_mask([current_inp_length],max([current_inp_length]))
        attentions = torch.zeros(max_len, 1, inp_embeddings.shape[1])
        trg_indexes = []
        for i in range(max_len):

            if i==0:
                trg_tensor = torch.LongTensor([output_vocab['<sos>']])
                
            with torch.no_grad():
                # input,hidden,encoder_outputs,encoder_length_mask
                output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

            attentions[i] = attention
            
            pred_token = output.argmax(1).item()
        
            trg_indexes.append(pred_token)
            trg_tensor = torch.LongTensor([pred_token])

            if pred_token == output_vocab['<eos>']:
                break
    
    trg_tokens = [output_index_to_word[i] for i in trg_indexes]
    return trg_tokens, attentions
    #print(trg_tokens)



In [34]:
def getmetrics(model,sentences,ground_truths):
    exact_match_count = 0
    per_word_matches = [0 for i in range(10)]

    for i in range(len(sentences)):

        trg_tokens, attention_weights = translate_sentence(sentences[i],model)
        ground_truth_tokens = list(ground_truths[i])

        exact_match = True
        for i in range(len(ground_truth_tokens)):

            if trg_tokens[i] == ground_truth_tokens[i]:
                per_word_matches[i]+=1
            else:
                exact_match = False
        
        if exact_match:
            exact_match_count+=1

    
    number = len(sentences)
    per_output_accuracy = [ 0.0 for i in range(10)]
    exact_match_accuracy = ((1.0*exact_match_count)/number)*100

    for i in range(len(per_word_matches)):

        per_output_accuracy[i] = ((1.0*per_word_matches[i]) / number) * 100

    averaged_1 = sum(per_output_accuracy)/10
    averaged_2 = (sum(per_word_matches)/(10*number))*100

    return exact_match_accuracy,per_output_accuracy,averaged_1,averaged_2

In [35]:
def return_test_dataset():
    ground_truth = []
    sentences = []
    with open("./Assignment4aTestDataset.txt",'r') as f:
        lines = f.readlines()
        for line in lines:
            nl_date , out_date = line.split(',')
            nl_date = nl_date.replace("\'","").strip()
            out_date = out_date.replace("\'","").strip()

            if "/" in nl_date:
                #number_list = list(nl_date)
                #nl_date = " ".join(number_list)
                split_on_slash = nl_date.split("/")
                nl_date = " / ".join(split_on_slash)

            nl_date = nl_date.lower()

            sentences.append(nl_date)
            ground_truth.append(out_date)
            
    return ground_truth,sentences

In [36]:
t = TranslationModel()
t.load_state_dict(torch.load('./translation_model_11'))



<All keys matched successfully>

In [37]:
output_tokens , attention = translate_sentence('21/04/2019',t.to(device_cpu))
print(output_tokens)

['2', '0', '1', '9', '-', '0', '4', '-', '2', '1']


In [38]:
ground_truth,sentences = return_test_dataset()
getmetrics(t,sentences,ground_truth)

(95.72,
 [99.45,
  97.0,
  99.64,
  98.72999999999999,
  100.0,
  99.68,
  98.99,
  100.0,
  98.9,
  98.28],
 99.067,
 99.06700000000001)

In [ ]:

def plot_heatmap(sentence, translation, attention):

    fig, ax = plt.subplots()
    heatmap = ax.pcolor(attention, cmap='viridis')

    nl_date = sentence.replace("\'","").strip()
    nl_date = nl_date.lower()

    if "/" in nl_date:
        #number_list = list(nl_date)
        #nl_date = " ".join(number_list)
        split_on_slash = nl_date.split("/")
        nl_date = " / ".join(split_on_slash)

    src = nl_date.split(" ")
    trg = list(translation)

    ax.set_xticklabels(src, minor=False, rotation='vertical')
    ax.set_yticklabels(trg, minor=False)

    # put the major ticks at the middle of each cell
    # and the x-ticks on top
    ax.xaxis.tick_top()
    ax.set_xticks(np.arange(attention.shape[1]) + 0.5, minor=False)
    ax.set_yticks(np.arange(attention.shape[0]) + 0.5, minor=False)
    ax.invert_yaxis()

    plt.colorbar(heatmap)
    plt.show()


In [ ]:
output_tokens , attention = translate_sentence('Saturday 29 February 2021',t.to(device_cpu))
plot_heatmap('Saturday 29 February 2021',''.join(output_tokens),attention.squeeze(1))

In [ ]:
output_tokens , attention = translate_sentence('29 February 2020',t.to(device_cpu))
plot_heatmap('9 February 2020',''.join(output_tokens),attention.squeeze(1))

In [ ]:
rnn = nn.GRU(100,64, num_layers= 2 ,batch_first= True ,bidirectional=True)
inp = torch.randn((2,8,100))
outputs,hidden = rnn(inp)

In [ ]:
outputs.shape

In [ ]:
hidden.shape

In [ ]:
inq = torch.cat([hidden[-i , : , :] for i in range(4)],dim=1)


In [ ]:
inq.shape

In [ ]:
rnn(inp)[1].shape

In [ ]:
embedding_layer = nn.Embedding(10,30)

embedding_layer(torch.tensor([[10,2,3],[4,5,6]])).shape